# Understanding Pandas `DataFrame` operations

In [1]:
import numpy as np
import pandas as pd

## `DataFrame` used for tests

Number of births for each name and year, with an imbalance for genre=F, so we can spot the operations more easily (would be harder to follow what the operations are doing if the F/M data were the same).

In [63]:
births = pd.DataFrame(np.array([
    [1970, 'Mary', 'F', 1],
    [1970, 'James', 'M', 2],
    [1970, 'John', 'M', 3],

    [1971, 'Mary', 'F', 10],
    [1971, 'James', 'M', 20],
    [1971, 'John', 'M', 30],

    ]), columns=('year', 'name', 'genre', 'births'))

births = births.astype({'year': 'int', 'births': 'int'})
births

,year,name,genre,births
0,1970,Mary,F,1
1,1970,James,M,2
2,1970,John,M,3
3,1971,Mary,F,10
4,1971,James,M,20
5,1971,John,M,30


## Group

Calculate number of births by genre in each year.

In [64]:
births_by_genre_year = births.groupby(['year','genre']).sum()
births_by_genre_year

births
year genre        
1970 F           1
     M           5
1971 F          10
     M          50

Although not wrong, it's not the easiest way to visualize the numbers. Move `genre` to columns with `unstack()`:

In [65]:
births_by_genre_year = births_by_genre_year.unstack()
births_by_genre_year

births    
genre      F   M
year            
1970       1   5
1971      10  50

Pivot table version - do it all in one shot.

In [66]:
births.pivot_table('births', index='year', columns='genre',
                   aggfunc='sum')

genre,F,M
year,,
1970,1,5
1971,10,50


## Slicing

Picking the `DataFrame` apart by addressing specific pieces of it.

Number of births for a specific year and genre.

In [67]:
births_by_genre_year.loc[1970]['births']['M']

5

Number of births for a specific genre, all years.

In [68]:
births_by_genre_year.loc[:]['births']['M']

year
1970     5
1971    50
Name: M, dtype: int64

## Aggregations - transform and augment data

**VERY IMPORTANT CONCEPT**: the axis specifies the dimension that will be collapsed (i.e. will be operated on and consolidated).

If we ask to perform an opeartion on rows, the rows are collapsed and aggreated by the specified operation.

### By rows: no axis, axis=0, axis='rows'

Collapse the rows and leave columns in place, i.e. aggregate (sums) row values.

In [69]:
# These are equivalent
births_by_genre_year.sum().unstack()
births_by_genre_year.sum(axis=0).unstack()
births_by_genre_year.sum(axis='rows').unstack()

genre,F,M
births,11,55


### By columns: axis=1, axis='columns'

Collapse the columns and leave the rows in place, i.e. aggregate (sums) colum values.

In [70]:
# These are equivalent
births_by_genre_year.sum(axis=1)
births_by_genre_year.sum(axis='columns')

year
1970     6
1971    60
dtype: int64

### `groupby`: split, apply, combine

#### Calculating the percentage of M/F in each year

How it works:

* **Split**: Since we want the percentage by year and by genre, we need to split using both year and genre.
* **Apply**: Once the dataset is split, we can apply the calculations we need to that slice.
* **Combine**: Pandas reassembles the results of the _apply_ step into one `DataFrame`.

In the code below, the `pct_genre_year(group)` is the _apply_ step. The `print(...)` statements are not needed for the calculations. They are there to help us visualize how the data set was split.

In [71]:
def pct_genre_year(group):
    print('\nThis is the split for {},{}'.format
          (group.year.iloc[0], group.genre.iloc[0]))
    print(group)

    group['pct_genre_year'] = group.births / group.births.sum() * 100
    return group

births_pct_year_genre = births.groupby(['year', 'genre']).apply(pct_genre_year)
births_pct_year_genre


This is the split for 1970,F
   year  name genre  births
0  1970  Mary     F       1

This is the split for 1970,M
   year   name genre  births
1  1970  James     M       2
2  1970   John     M       3

This is the split for 1971,F
   year  name genre  births
3  1971  Mary     F      10

This is the split for 1971,M
   year   name genre  births
4  1971  James     M      20
5  1971   John     M      30


,year,name,genre,births,pct_genre_year
0,1970,Mary,F,1,100.0
1,1970,James,M,2,40.0
2,1970,John,M,3,60.0
3,1971,Mary,F,10,100.0
4,1971,James,M,20,40.0
5,1971,John,M,30,60.0


Visually:

![Split, apply, combine example](images/split_apply_combine_births.png)

#### Calculating number of births per year

We do not need to return the same number of rows when grouping a dataset.

In this example we consolidate the M and F columns to get an overall count of births per year, independent of genre.

In [72]:
def births_per_year(group):
    return group['births'].sum()

births.groupby(['year']).apply(births_per_year)

year
1970     6
1971    60
dtype: int64

When the `apply` function is simple, we can use a lambda expression:

In [73]:
births.groupby(['year']).apply(lambda x: x['births'].sum())

year
1970     6
1971    60
dtype: int64

We can clean up the syntax a bit:

* Remove the array from `groupby`, since we are grouping by one column only
* Use the column name `births` as a variable name

In [74]:
births.groupby('year').apply(lambda x: x.births.sum())

year
1970     6
1971    60
dtype: int64

Finally, `pivot_table` may be a clearer alternative when consolidating information.

In [75]:
births.pivot_table('births', index='year', aggfunc='sum')

,births
year,
1970,6
1971,60


The `pivot_table` above works for this case because we ommited the `columns` parameter. If we add it back in, we break the results down further (as we used above, in the [group example](#Group)).

In [76]:
births.pivot_table('births', index='year', columns='genre',
                   aggfunc='sum')

genre,F,M
year,,
1970,1,5
1971,10,50


**************
TODO: differences between groupby and pivot_table, e.g. Series vs. DataFrame. Other guidelines?
**************

## Querying - asking questions and getting answers

* How many boys born overall? <-- save as filter
* How many girls born overall? <-- use ~
* How many boys born in 1970?
* How many people named Mary born in 1970? <--- use eval
* How many names used more than ten times in one year?
* What is the average number of births per year?
* What is the average number of girls born per year?
* What is the year with the highest number of births?
* What is the year with the lowest number of girl births?

In [77]:
births[births['genre'] == 'M']['births'].sum()

55

In [78]:
births.query('genre == "M"').births.sum()

55

**************
TODO: when to use query vs. regular filtering
**************

## A word about `value_counts`

If we are interested only in the count or proportion of categorical columns, we can use `values_counts`:

In [ ]:
births['genre'].value_counts()

In [ ]:
births['genre'].value_counts(normalize=True)